In [1]:
import json
import time
import numpy as np
import paho.mqtt.client as mqtt
import stable_baselines3
import os

/opt/conda/envs/conda_environment/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
class MyController:
    def __init__(self, radTrig=100, TinTrig=24):
        self.radTrig = radTrig
        self.TinTrig = TinTrig

    def compute_action(self, measurements: dict) -> dict:
        """
        measurements expected to include:
        'T_ext', 'DNI', 'Tin_Bathroom1', 'Tin_Room1',
        'Tin_Bathroom', 'Tin_Livingroom', 'Tin_Room', 'Tin_Kitchen'
        """

        T_ext = float(measurements.get('T_ext', 0.0))
        Rad = float(measurements.get('DNI', 0.0))

        Tin_Bathroom1  = float(measurements.get('Tin_Bathroom1', 0.0))
        Tin_Room1      = float(measurements.get('Tin_Room1', 0.0))
        Tin_Bathroom   = float(measurements.get('Tin_Bathroom', 0.0))
        Tin_Livingroom = float(measurements.get('Tin_Livingroom', 0.0))
        Tin_Room       = float(measurements.get('Tin_Room', 0.0))
        Tin_Kitchen    = float(measurements.get('Tin_Kitchen', 0.0))  # 👈 assumed name

        # 🔽 Actuator names
        payload = {
            'Kitchen_shade_FMU': 0.0,
            'Bath_shade_1_FMU': 0.0,
            'Room1_shade_1_FMU': 0.0,
            'Room1_shade_2_FMU': 0.0,
            'Bath2_Shade_FMU': 0.0,
            'Living_shade_1_FMU': 0.0,
            'Living_shade_2_FMU': 0.0,
            'Room2_shade1_FMU': 0.0,
            'Room2_shade_2_FMU': 0.0
        }

        # 🛠 Same rule everywhere: if Tin > TinTrig and Rad > radTrig → 7.0 else 0.0

        # Kitchen
        if Tin_Kitchen > self.TinTrig and Rad > self.radTrig:
            payload['Kitchen_shade_FMU'] = 7.0

        # Bathroom1
        if Tin_Bathroom1 > self.TinTrig and Rad > self.radTrig:
            payload['Bath_shade_1_FMU'] = 7.0

        # Bathroom2
        if Tin_Bathroom > self.TinTrig and Rad > self.radTrig:
            payload['Bath2_Shade_FMU'] = 7.0

        # Room1 → two shades
        if Tin_Room1 > self.TinTrig and Rad > self.radTrig:
            payload['Room1_shade_1_FMU'] = 7.0
            payload['Room1_shade_2_FMU'] = 7.0

        # Livingroom → two shades
        if Tin_Livingroom > self.TinTrig and Rad > self.radTrig:
            payload['Living_shade_1_FMU'] = 7.0
            payload['Living_shade_2_FMU'] = 7.0

        # Room2 → two shades
        if Tin_Room > self.TinTrig and Rad > self.radTrig:
            payload['Room2_shade1_FMU'] = 7.0
            payload['Room2_shade_2_FMU'] = 7.0

        return payload

In [3]:
# 📦 Buffer for last observation
obs_buffer = {}


# 🧾 MQTT callbacks (solo qui, non nel controller)
def on_connect(client, userdata, flags, rc):
    print(f"✅ Connected to MQTT broker (rc={rc})")
    client.subscribe("building/observation", qos=1)
    print("✅ Subscribed to topic: building/observation")

def on_obs(client, userdata, msg):
    global obs_buffer
    try:
        data = json.loads(msg.payload)
        obs_buffer.update(data)
    except Exception as e:
        print(f"Error parsing observation: {e}")


# 🧩 Setup MQTT client (usando le env var come nel controller originale)
mqtt_broker = os.getenv('MQTT_BROKER_HOST', 'mosquitto')
mqtt_port = int(os.getenv('MQTT_BROKER_PORT', 1883))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_obs
client.connect(mqtt_broker, mqtt_port)
client.loop_start()

# 🔧 Istanza del controller puro
controller = MyController()

/tmp/ipykernel_777/613007330.py:24: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


✅ Connected to MQTT broker (rc=0)
✅ Subscribed to topic: building/observation


In [4]:
# 🔁 Main loop
try:
    while True:
        if obs_buffer:
            # We directly pass the entire observation dictionary to the controller.
            # The controller is expected to return an action dict.
            action_dict = controller.compute_action(obs_buffer)

            # Publish the action to the MQTT broker.
            # You may switch to topic "/actuator" if you want to preserve the old behavior.
            client.publish("building/action", json.dumps(action_dict), qos=1)
            print(f"[DEBUG] Action sent: {action_dict}")

            # Clear the observation buffer so that we wait for a new observation
            # before computing the next action.
            obs_buffer.clear()

        # Small sleep to avoid busy-looping while waiting for new messages.
        time.sleep(0.001)

except KeyboardInterrupt:
    # Graceful shutdown on Ctrl+C
    print("Stopping...")
    client.loop_stop()
    client.disconnect()


[DEBUG] Action sent: {'Kitchen_shade_FMU': 0.0, 'Bath_shade_1_FMU': 0.0, 'Room1_shade_1_FMU': 0.0, 'Room1_shade_2_FMU': 0.0, 'Bath2_Shade_FMU': 0.0, 'Living_shade_1_FMU': 0.0, 'Living_shade_2_FMU': 0.0, 'Room2_shade1_FMU': 0.0, 'Room2_shade_2_FMU': 0.0}
[DEBUG] Action sent: {'Kitchen_shade_FMU': 0.0, 'Bath_shade_1_FMU': 0.0, 'Room1_shade_1_FMU': 0.0, 'Room1_shade_2_FMU': 0.0, 'Bath2_Shade_FMU': 0.0, 'Living_shade_1_FMU': 0.0, 'Living_shade_2_FMU': 0.0, 'Room2_shade1_FMU': 0.0, 'Room2_shade_2_FMU': 0.0}
[DEBUG] Action sent: {'Kitchen_shade_FMU': 0.0, 'Bath_shade_1_FMU': 0.0, 'Room1_shade_1_FMU': 0.0, 'Room1_shade_2_FMU': 0.0, 'Bath2_Shade_FMU': 0.0, 'Living_shade_1_FMU': 0.0, 'Living_shade_2_FMU': 0.0, 'Room2_shade1_FMU': 0.0, 'Room2_shade_2_FMU': 0.0}
[DEBUG] Action sent: {'Kitchen_shade_FMU': 0.0, 'Bath_shade_1_FMU': 0.0, 'Room1_shade_1_FMU': 0.0, 'Room1_shade_2_FMU': 0.0, 'Bath2_Shade_FMU': 0.0, 'Living_shade_1_FMU': 0.0, 'Living_shade_2_FMU': 0.0, 'Room2_shade1_FMU': 0.0, 'Room2_sha